In [1]:
import numpy as np
import pandas as pd
import os
import re
import pickle
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['VISIBLE_CUDA_DEVICES'] = ""
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ./PETA/HRPInception/

thr3_attributes_list.txt
thr3_v1_epoch001_valloss0.647757.hdf5_results.file
thr3_v1_epoch002_valloss0.663170.hdf5_results.file
thr3_v1_epoch003_valloss0.727662.hdf5_results.file
thr3_v1_epoch004_valloss0.794733.hdf5_results.file
thr3_v1_epoch006_valloss0.818306.hdf5_results.file
thr3_v1_epoch008_valloss0.869249.hdf5_results.file
thr3_v1_epoch009_valloss0.880410.hdf5_results.file
thr3_v1_epoch012_valloss0.914101.hdf5_results.file
thr3_v1_epoch014_valloss0.930108.hdf5_results.file
thr3_v1_epoch016_valloss0.942539.hdf5_results.file
thr3_v1_epoch018_valloss0.953733.hdf5_results.file
thr3_v1_epoch020_valloss0.960704.hdf5_results.file
thr3_v1_epoch021_valloss0.967176.hdf5_results.file
thr3_v1_epoch028_valloss0.971832.hdf5_results.file
thr3_v1_epoch029_valloss0.973245.hdf5_results.file
thr3_v1_epoch031_valloss0.974265.hdf5_results.file
thr3_v1_epoch033_valloss0.975990.hdf5_results.file
thr3_v1_epoch034_valloss0.976939.hdf5_results.file
thr3_v1_epoch035_valloss0.979274.hdf5_results.file
thr3_v

In [2]:
labels_list_file = r"/home/anhaoran/data/pedestrian_attributes_PETA/PETA/labels.txt" 
labels_list_data = open(labels_list_file)
lines = labels_list_data.readlines()
attributes_list = []
for line in lines:
    data = line.split()
    #print(attr)
    attributes_list.append(data[1])
print("The total attributes labeled: ", len(attributes_list))
print(attributes_list)

The total attributes labeled:  65
['accessoryHeadphone', 'personalLess15', 'personalLess30', 'personalLess45', 'personalLess60', 'personalLarger60', 'carryingBabyBuggy', 'carryingBackpack', 'hairBald', 'footwearBoots', 'lowerBodyCapri', 'carryingOther', 'carryingShoppingTro', 'carryingUmbrella', 'lowerBodyCasual', 'upperBodyCasual', 'personalFemale', 'carryingFolder', 'lowerBodyFormal', 'upperBodyFormal', 'accessoryHairBand', 'accessoryHat', 'lowerBodyHotPants', 'upperBodyJacket', 'lowerBodyJeans', 'accessoryKerchief', 'footwearLeatherShoes', 'upperBodyLogo', 'hairLong', 'lowerBodyLongSkirt', 'upperBodyLongSleeve', 'lowerBodyPlaid', 'lowerBodyThinStripes', 'carryingLuggageCase', 'personalMale', 'carryingMessengerBag', 'accessoryMuffler', 'accessoryNothing', 'carryingNothing', 'upperBodyNoSleeve', 'upperBodyPlaid', 'carryingPlasticBags', 'footwearSandals', 'footwearShoes', 'hairShort', 'lowerBodyShorts', 'upperBodyShortSleeve', 'lowerBodyShortSkirt', 'footwearSneakers', 'footwearStockin

In [3]:
low = [27,32,50,56]
hs = [0,8,20,21,25,28,36,37,44,54]
ub = [23,30,39,46,51,55,58,59,60]
lb = [22,24,29,45,47,53,57]
sh = [9,26,42,43,48,49]
at = [6,7,11,12,13,17,33,35,38,41,52]
high_whole = [1,2,3,4,5,16,34]
high_ub = [15,19,40]
high_lb = [10,14,18,31]
parts = [[len(low)], [len(hs), len(ub), len(lb), len(sh), len(at)], [len(high_whole), len(high_ub), len(high_lb)]]
idx_indices = list(np.hstack((low, hs, ub, lb, sh, at, high_whole, high_ub, high_lb)))
attributes_list_hiar = list(np.array(attributes_list)[idx_indices])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)
data_dict = {"attributes": attributes_list_hiar}

The total attributes labeled:  61
['upperBodyLogo', 'lowerBodyThinStripes', 'upperBodyThinStripes', 'upperBodyThickStripes', 'accessoryHeadphone', 'hairBald', 'accessoryHairBand', 'accessoryHat', 'accessoryKerchief', 'hairLong', 'accessoryMuffler', 'accessoryNothing', 'hairShort', 'accessorySunglasses', 'upperBodyJacket', 'upperBodyLongSleeve', 'upperBodyNoSleeve', 'upperBodyShortSleeve', 'upperBodySuit', 'upperBodySweater', 'upperBodyTshirt', 'upperBodyOther', 'upperBodyVNeck', 'lowerBodyHotPants', 'lowerBodyJeans', 'lowerBodyLongSkirt', 'lowerBodyShorts', 'lowerBodyShortSkirt', 'lowerBodySuits', 'lowerBodyTrousers', 'footwearBoots', 'footwearLeatherShoes', 'footwearSandals', 'footwearShoes', 'footwearSneakers', 'footwearStocking', 'carryingBabyBuggy', 'carryingBackpack', 'carryingOther', 'carryingShoppingTro', 'carryingUmbrella', 'carryingFolder', 'carryingLuggageCase', 'carryingMessengerBag', 'carryingNothing', 'carryingPlasticBags', 'carryingSuitcase', 'personalLess15', 'personalLe

In [4]:
data_root = "/home/anhaoran/data/pedestrian_attributes_PETA/PETA/"
data = np.array(pd.read_csv(data_root + "PETA.csv"))[:, 1:]
    
labels_list_data = open(data_root + "labels.txt")
lines = labels_list_data.readlines()

y_data = []
for i in range(len(data)):
    y_data.append(data[i, 1:])
y_data = np.asarray(y_data)
data_y = y_data[11400:].astype("float32")
data_y_hiar = data_y[:, idx_indices].astype("float32")
y_test = data_y
y_test_hiar = data_y_hiar
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)

The shape of the y_test is:  (7600, 65)
The shape of the y_test is:  (7600, 61)


In [5]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0.).count(True)
        #print(TP, P, TN, N)
        #print(P,',', N,',', TP,',', TN)
        #if P != 0:
        res += TP/P + TN/N
        """else:
            res += 1 + TN/N"""
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            #print(y_pred[i])
            #print(y_true[i])
            #print(sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i])))
            M_ += 1
    #print(M_)
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

In [6]:
def calculate_accuracy(gt_result, pt_result):
    ''' obtain the label-based and instance-based accuracy '''
    # compute the label-based accuracy
    if gt_result.shape != pt_result.shape:
        print('Shape beteen groundtruth and predicted results are different')
    # compute the label-based accuracy
    result = {}
    gt_pos = np.sum((gt_result == 1).astype(float), axis=0)
    gt_neg = np.sum((gt_result == -1).astype(float), axis=0)
    pt_pos = np.sum((gt_result == 1).astype(float) * (pt_result == 1).astype(float), axis=0)
    pt_neg = np.sum((gt_result == -1).astype(float) * (pt_result == -1).astype(float), axis=0)
    label_pos_acc = 1.0*pt_pos/gt_pos
    label_neg_acc = 1.0*pt_neg/gt_neg
    label_acc = (label_pos_acc + label_neg_acc)/2
    result['label_pos_acc'] = label_pos_acc
    result['label_neg_acc'] = label_neg_acc
    result['label_acc'] = label_acc
    # compute the instance-based accuracy
    # precision
    gt_pos = np.sum((gt_result == 1).astype(float), axis=1)
    pt_pos = np.sum((pt_result == 1).astype(float), axis=1)
    floatersect_pos = np.sum((gt_result == 1).astype(float)*(pt_result == 1).astype(float), axis=1)
    union_pos = np.sum(((gt_result == 1)+(pt_result == 1)).astype(float),axis=1)
    # avoid empty label in predicted results
    cnt_eff = float(gt_result.shape[0])
    for iter, key in enumerate(gt_pos):
        if key == 0:
            union_pos[iter] = 1
            pt_pos[iter] = 1
            gt_pos[iter] = 1
            cnt_eff = cnt_eff - 1
            continue
        if pt_pos[iter] == 0:
            pt_pos[iter] = 1
    instance_acc = np.sum(floatersect_pos/union_pos)/cnt_eff
    instance_precision = np.sum(floatersect_pos/pt_pos)/cnt_eff
    instance_recall = np.sum(floatersect_pos/gt_pos)/cnt_eff
    floatance_F1 = 2*instance_precision*instance_recall/(instance_precision+instance_recall)
    result['instance_acc'] = instance_acc
    result['instance_precision'] = instance_precision
    result['instance_recall'] = instance_recall
    result['instance_F1'] = floatance_F1
    return result

In [7]:
import sys
import keras.backend as K

def keras_mA(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    true_negatives = np.sum(np.round(np.clip((1 - y_true) * (1 - y_pred), 0, 1)), axis=-1)
    possible_negatives = np.sum(np.round(np.clip(1 - y_true, 0, 1)), axis=-1)
    mean_acc = (true_positives / (possible_positives + sys.float_info.epsilon) + true_negatives / (possible_negatives + sys.float_info.epsilon)) / 2
    return mean_acc

def keras_acc(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    possible_positives += np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    possible_positives -= true_positives
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return np.average(mean_acc)

def keras_prec(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return np.average(mean_acc)

def keras_rec(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return np.average(mean_acc)


def keras_acc_instance(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    possible_positives += np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    possible_positives -= true_positives
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return mean_acc

def keras_prec_instance(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_pred, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return mean_acc

def keras_rec_instance(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)), axis=-1)
    mean_acc = true_positives / (possible_positives + sys.float_info.epsilon)
    return mean_acc

def keras_K_mA(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=-1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=-1)
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)), axis=-1)
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)), axis=-1)
    mean_acc = (true_positives / (possible_positives + K.epsilon()) + true_negatives / (possible_negatives + K.epsilon())) / 2
    return mean_acc

Using TensorFlow backend.


In [8]:
def mA_thr(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true[:])
    N = M - P
    TP = sum(y_pred[:]*y_true[:])
    TN = list(y_pred[:]+y_true[:] == 0).count(True)
    if P != 0:
        res += TP/P + TN/N
    else:
        res += TN/N
    return res / 2


def find_best_fixed_threshold(preds, targs, do_plot=False):
    score = []
    thrs = np.arange(0, 1, 0.001)
    for thr in thrs:
        ### thr1/2
        #score.append(my_f2(targs, (preds > thr).astype(int) ))
        ### thr3
        score.append(mA_thr((preds[:, 0] > thr).astype(int), targs[:, 0]) )
        #res = calculate_accuracy(np.sign((preds[:] - thr)), np.sign(targs[:] - 0.5)) 
        #score.append(np.sum(res['label_acc'])/len(res['label_acc']))
    score = np.array(score)
    pm = score.argmax()
    best_thr, best_score = thrs[pm], score[pm].item()
    if do_plot:
        print('thr={best_thr:.9f}'.format(best_thr=best_thr), 'F2={best_score:.3f}'.format(best_score=best_score))
        plt.plot(thrs, score)
        plt.vlines(x=best_thr, ymin=score.min(), ymax=score.max())
        plt.text(best_thr+0.03, best_score-0.01, '$F_{2}=${best_score:.3f}'.format(best_score=best_score), fontsize=14);
        plt.show()
    return best_thr, best_score

In [9]:
reg = "HRPInception_PETA_v*"
weights = [s for s in os.listdir("../results/predictions/") 
          if re.match(reg, s)]
print(weights)

for w in weights:
    predictions_prob = np.load(
            "../results/predictions/" + w)
    print(w)
    
    predictions = predictions_prob
    y_test_hiar = data_y_hiar
    label = y_test_hiar
    results = keras_mA(label, predictions)
    #print(results)
    #print(results.shape)
    print(np.average(results))
    k_acc = keras_acc(label, predictions)
    k_prec = keras_prec(label, predictions)
    k_rec = keras_rec(label, predictions)
    print(k_acc)
    print(k_prec)
    print(k_rec)
    print(2*k_prec*k_rec/(k_prec + k_rec))
    
    predictions = np.sign(predictions_prob-0.5)#[:, selected]
    y_test_hiar = data_y_hiar
    #"""
    best_thr = np.zeros((1, 61))
    for i in range(61):
        best_thr[0, i], _ = find_best_fixed_threshold(predictions_prob[:, i:i+1], y_test_hiar[:, i:i+1])
    predictions = np.sign(predictions_prob - best_thr)
    #"""
    label = np.sign(y_test_hiar-0.5)#[:, selected]
    results = calculate_accuracy(label, predictions)
    print(results)
    print(np.sum(results['label_acc'])/len(results['label_acc']))

['HRPInception_PETA_v2_epoch028_valloss0.963230.hdf5.npy', 'HRPInception_PETA_v2_epoch494_valloss0.986102.hdf5.npy', 'HRPInception_PETA_v2_epoch025_valloss0.962013.hdf5.npy', 'HRPInception_PETA_v2_epoch102_valloss0.984418.hdf5.npy', 'HRPInception_PETA_v1_epoch100_valloss0.984540.hdf5.npy', 'HRPInception_PETA_v2_epoch342_valloss0.985735.hdf5.npy', 'HRPInception_PETA_v2_epoch024_valloss0.955659.hdf5.npy', 'HRPInception_PETA_v2_epoch017_valloss0.924177.hdf5.npy', 'HRPInception_PETA_v1_epoch310_valloss0.985879.hdf5.npy', 'HRPInception_PETA_v2_epoch082_valloss0.983475.hdf5.npy', 'HRPInception_PETA_v1_epoch069_valloss0.984234.hdf5.npy', 'HRPInception_PETA_v1_epoch254_valloss0.985722.hdf5.npy', 'HRPInception_PETA_v1_epoch331_valloss0.986157.hdf5.npy', 'HRPInception_PETA_v2_epoch005_valloss0.807957.hdf5.npy', 'HRPInception_PETA_v1_epoch009_valloss0.880410.hdf5.npy', 'HRPInception_PETA_v2_epoch032_valloss0.966221.hdf5.npy', 'HRPInception_PETA_v1_epoch031_valloss0.974265.hdf5.npy', 'HRPInception

/home/anhaoran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


{'label_pos_acc': array([0.88157895,        nan, 0.921875  , 1.        , 0.96      ,
       0.98895028, 0.9483871 , 0.96465222, 0.92682927, 0.95648232,
       0.95419847, 0.9823441 , 0.96229632, 0.95094937, 0.93055556,
       0.9842924 , 0.96116505, 0.93442623, 0.96969697, 0.94444444,
       0.94842407, 0.92885193, 1.        , 0.98387097, 0.95910781,
       0.94805195, 0.98540146, 0.93288591, 0.98126464, 0.96473552,
       0.96811594, 0.96220372, 0.96410256, 0.93986384, 0.95564516,
       0.93119266, 0.96634615, 0.94195804, 0.87456647, 1.        ,
       0.87096774, 0.95287958, 0.93023256, 0.85657371, 0.87430309,
       0.90442478, 0.92405063, 1.        , 0.97149123, 0.94651927,
       0.9532294 , 0.97851562, 0.97161895, 0.97150917, 0.97215813,
       0.9804287 , 0.95714286, 0.95294118, 0.98256881, 0.97129187,
       0.5       ]), 'label_neg_acc': array([0.96388292, 1.        , 0.98832272, 0.99022328, 0.93152318,
       0.99191266, 0.95720165, 0.97248252, 1.        , 0.98053393,
      

{'label_pos_acc': array([0.875     ,        nan, 0.90625   , 1.        , 0.86      ,
       0.97237569, 0.9516129 , 0.97947548, 0.92682927, 0.97778785,
       0.97900763, 0.99286251, 0.98105343, 0.96993671, 0.95039683,
       0.97689171, 0.94174757, 0.96135831, 0.97668998, 0.93518519,
       0.95272206, 0.97274197, 0.94117647, 0.98387097, 0.97191243,
       0.95670996, 0.94890511, 0.94295302, 0.98360656, 0.97707809,
       0.97101449, 0.98174247, 0.96923077, 0.95537065, 0.97177419,
       0.94954128, 0.99038462, 0.97132867, 0.93352601, 0.75      ,
       0.87096774, 0.95811518, 0.96511628, 0.94322709, 0.91485048,
       0.94513274, 0.93037975, 1.        , 0.98574561, 0.98147595,
       0.97327394, 0.98632812, 0.9792695 , 0.98843441, 0.9938471 ,
       0.98508854, 0.91428571, 0.91764706, 0.99296636, 0.98564593,
       0.5       ]), 'label_neg_acc': array([0.96482277, 1.        , 0.99442675, 0.98123927, 0.99655629,
       0.9925866 , 0.98079561, 0.99330656, 1.        , 0.98461253,
      

{'label_pos_acc': array([0.88815789,        nan, 0.921875  , 1.        , 0.96      ,
       0.96132597, 0.93870968, 0.96351197, 0.92682927, 0.9655485 ,
       0.95038168, 0.9567994 , 0.95433876, 0.95886076, 0.91468254,
       0.96662136, 0.96116505, 0.93442623, 0.97202797, 0.92592593,
       0.95415473, 0.93185493, 0.88235294, 0.98387097, 0.94671623,
       0.95670996, 0.97080292, 0.93288591, 0.9882904 , 0.96120907,
       0.96521739, 0.94746957, 0.91794872, 0.9183056 , 0.96451613,
       0.94954128, 0.98557692, 0.92167832, 0.82716763, 1.        ,
       0.87096774, 0.94764398, 0.95348837, 0.79930279, 0.8540294 ,
       0.8920354 , 0.9556962 , 0.98823529, 0.97916667, 0.9629519 ,
       0.95100223, 0.98242188, 0.9760612 , 0.97884344, 0.98000308,
       0.95992544, 0.92857143, 0.90588235, 0.98501529, 0.9645933 ,
       1.        ]), 'label_neg_acc': array([0.95287325, 1.        , 0.9669586 , 0.9985467 , 0.90754967,
       0.98881251, 0.98408779, 0.97590361, 1.        , 0.96292176,
      

...

{'label_pos_acc': array([0.89473684,        nan, 0.890625  , 1.        , 0.86      ,
       0.97790055, 0.94193548, 0.96693273, 0.92682927, 0.97824116,
       0.96564885, 0.98703982, 0.97783251, 0.96993671, 0.92063492,
       0.97371998, 0.99029126, 0.95901639, 0.96736597, 0.93518519,
       0.9469914 , 0.96465696, 0.88235294, 0.98387097, 0.95621644,
       0.95670996, 0.96350365, 0.92281879, 0.9765808 , 0.9768262 ,
       0.96231884, 0.97341448, 0.94358974, 0.95272315, 0.95403226,
       0.94495413, 0.99038462, 0.96573427, 0.87919075, 0.75      ,
       0.87096774, 0.94764398, 0.96511628, 0.90836653, 0.90369995,
       0.93628319, 0.9556962 , 1.        , 0.98464912, 0.97042127,
       0.9844098 , 0.9921875 , 0.98124383, 0.97771509, 0.9924627 ,
       0.97576887, 0.95714286, 0.90588235, 0.98746177, 0.97990431,
       0.5       ]), 'label_neg_acc': array([0.9575725 , 1.        , 0.98301486, 0.99762188, 0.9989404 ,
       0.98800377, 0.9739369 , 0.9903317 , 0.99042298, 0.97868001,
      

{'label_pos_acc': array([0.90789474,        nan, 0.921875  , 1.        , 0.86      ,
       0.95027624, 0.92258065, 0.97149373, 0.92682927, 0.97597461,
       0.95801527, 0.9810293 , 0.97821144, 0.95727848, 0.93849206,
       0.97885516, 0.96116505, 0.95081967, 0.96969697, 0.93518519,
       0.96848138, 0.96789097, 0.97058824, 0.98387097, 0.96902107,
       0.94805195, 0.95620438, 0.9261745 , 0.98126464, 0.96725441,
       0.95362319, 0.98206278, 0.96923077, 0.94818457, 0.96532258,
       0.97706422, 0.99038462, 0.96993007, 0.89479769, 1.        ,
       0.87096774, 0.94764398, 0.96511628, 0.91334661, 0.89761784,
       0.95044248, 0.93037975, 0.98823529, 0.98940058, 0.97460412,
       0.97327394, 0.984375  , 0.97680158, 0.97686883, 0.99292417,
       0.96458527, 0.92857143, 0.90588235, 0.99082569, 0.97129187,
       0.5       ]), 'label_neg_acc': array([0.96267454, 1.        , 0.98022824, 0.99894306, 0.99456954,
       0.99973042, 0.99163237, 0.98869552, 0.99986699, 0.97812384,
      

{'label_pos_acc': array([0.88157895,        nan, 0.890625  , 1.        , 0.86      ,
       0.96685083, 0.92580645, 0.97833523, 0.92682927, 0.98186763,
       0.98091603, 0.99192337, 0.98237969, 0.97943038, 0.94642857,
       0.98610482, 0.94174757, 0.96252927, 0.97668998, 0.91666667,
       0.96704871, 0.97089397, 0.88235294, 0.98387097, 0.97149938,
       0.95670996, 0.96350365, 0.9295302 , 0.98594848, 0.97581864,
       0.96811594, 0.97277386, 0.95897436, 0.95763994, 0.97903226,
       0.94495413, 0.98557692, 0.97132867, 0.94913295, 0.75      ,
       0.87096774, 0.94240838, 0.97674419, 0.94870518, 0.9401926 ,
       0.9539823 , 0.94303797, 1.        , 0.9872076 , 0.9737078 ,
       0.98218263, 0.99023438, 0.97951629, 0.98222849, 0.99369328,
       0.97856477, 0.94285714, 0.90588235, 0.99296636, 0.97607656,
       0.5       ]), 'label_neg_acc': array([0.97086466, 1.        , 0.99694798, 0.99947153, 0.9994702 ,
       0.99555196, 0.99080933, 0.99330656, 1.        , 0.98090471,
      

{'label_pos_acc': array([0.86842105,        nan, 0.890625  , 1.        , 0.86      ,
       0.96685083, 0.93548387, 0.97833523, 0.92682927, 0.98232094,
       0.97519084, 0.98948159, 0.98503221, 0.97151899, 0.93650794,
       0.98534965, 0.91262136, 0.96370023, 0.98135198, 0.90740741,
       0.98137536, 0.97043197, 0.88235294, 0.98387097, 0.97108633,
       0.95670996, 0.94160584, 0.94295302, 0.98360656, 0.97279597,
       0.96521739, 0.98078155, 0.95384615, 0.95499244, 0.9766129 ,
       0.9587156 , 0.98557692, 0.97762238, 0.95260116, 1.        ,
       0.87096774, 0.95811518, 0.96511628, 0.95318725, 0.96046629,
       0.95929204, 0.94303797, 1.        , 0.99013158, 0.9749029 ,
       0.97995546, 0.98632812, 0.98469891, 0.98251058, 0.9938471 ,
       0.98788444, 0.9       , 0.90588235, 0.99464832, 0.98373206,
       0.5       ]), 'label_neg_acc': array([0.98442535, 1.        , 0.99827495, 0.99947153, 0.99854305,
       0.99892169, 0.98710562, 0.99479399, 1.        , 0.98609566,
      

{'label_pos_acc': array([0.86184211,        nan, 0.921875  , 0.96774194, 1.        ,
       0.9558011 , 0.9       , 0.88483466, 0.92682927, 0.90435177,
       0.94274809, 0.92599549, 0.91644562, 0.90506329, 0.93055556,
       0.94698686, 0.93203883, 0.93793911, 0.96037296, 0.94444444,
       0.95558739, 0.81381381, 0.88235294, 0.95967742, 0.94423792,
       0.94805195, 0.95620438, 0.94630872, 0.94145199, 0.86246851,
       0.94202899, 0.84497117, 0.89230769, 0.83472012, 0.88951613,
       0.92201835, 0.99038462, 0.92517483, 0.77745665, 1.        ,
       0.93548387, 0.93717277, 0.97674419, 0.83416335, 0.83527623,
       0.90973451, 0.96202532, 0.95294118, 0.92361111, 0.91544667,
       0.91202673, 0.97070312, 0.93558736, 0.96332863, 0.96446701,
       0.94408201, 0.97142857, 0.92941176, 0.93287462, 0.9569378 ,
       1.        ]), 'label_neg_acc': array([0.95985499, 1.        , 0.97584926, 0.99696129, 0.98172185,
       0.982747  , 0.9733882 , 0.92235609, 0.99853684, 0.91490545,
      

In [11]:
all_data = pd.DataFrame()
root = "./PETA"
for dirpath, dirnames, filenames in os.walk(root):
    for filepath in filenames:
        if filepath.endswith(".txt"):
            continue
        file = open(os.path.join(dirpath, filepath), "rb")
        data = pickle.load(file)
        if "label_pos_acc" in data.keys():
            data.pop("label_pos_acc")
            data.pop("label_neg_acc")
            data.pop("label_acc")
        if filepath.find("_valloss") != -1:
            data["model"] = dirpath[dirpath.rindex("/")+1:] + "_" + filepath[:filepath.index("_valloss")]
        else:
            data["model"] = dirpath[dirpath.rindex("/")+1:] + "_" + filepath[:filepath.index("_model")]
        """
        # all_data.update(data)
        for key, value in data.items():
            if key not in all_data.keys():
                all_data.setdefault(key, [])
            all_data[key].append(value)
            """
        if filepath.find("_valloss") != -1:
            data["ma"] = float(filepath[filepath.rindex("valloss")+7:filepath.rindex(".h")])
        else:
            data["ma"] = 0
        data = pd.DataFrame(data, index=[0])
        all_data = all_data.append(data)

        
order = ["model", "label_ma", "ma", "instance_acc", "instance_precision", "instance_recall", "instance_F1"]
all_data = all_data[order]
all_data.sort_values(by="label_ma", ascending=False)

,model,label_ma,ma,instance_acc,instance_precision,instance_recall,instance_F1
0,HRPInception_thr3_v2_epoch462,0.962327,0.985864,0.963881,0.973158,0.980056,0.976595
0,HRPInception_thr3_v2_epoch490,0.962213,0.985942,0.964149,0.973271,0.980085,0.976666
0,HRPInception_thr3_v2_epoch428,0.961779,0.985811,0.963919,0.973582,0.979565,0.976564
0,HRPInception_thr3_v2_final500iter,0.961658,0.000000,0.964422,0.973525,0.980272,0.976887
0,HRPInception_thr3_v2_epoch494,0.961622,0.986102,0.963956,0.973420,0.980077,0.976737
0,HRPInception_thr3_v1_epoch049,0.961118,0.982766,0.944419,0.954023,0.977531,0.965634
0,HRPInception_thr3_v1_epoch035,0.960887,0.979274,0.929022,0.940004,0.975034,0.957199
0,HRPInception_thr3_v2_epoch097,0.960861,0.984057,0.950655,0.961001,0.977890,0.969372
0,HRPInception_thr3_v1_epoch039,0.960640,0.980840,0.935925,0.946513,0.976110,0.961083
0,HRPInception_thr3_v1_epoch308,0.960392,0.985731,0.963278,0.971305,0.980936,0.976096


In [12]:
all_data.sort_values(by="ma", ascending=False)

,model,label_ma,ma,instance_acc,instance_precision,instance_recall,instance_F1
0,HRPInception_thr3_v1_epoch331,0.959644,0.986157,0.965063,0.973479,0.981239,0.977343
0,HRPInception_thr3_v2_epoch494,0.961622,0.986102,0.963956,0.973420,0.980077,0.976737
0,HRPInception_thr3_v1_epoch330,0.959878,0.986043,0.965045,0.973430,0.981311,0.977354
0,HRPInception_thr3_v1_epoch312,0.957911,0.986015,0.964828,0.973485,0.980401,0.976931
0,HRPInception_thr3_v2_epoch490,0.962213,0.985942,0.964149,0.973271,0.980085,0.976666
0,HRPInception_thr3_v1_epoch310,0.960328,0.985879,0.964570,0.973248,0.980563,0.976892
0,HRPInception_thr3_v2_epoch462,0.962327,0.985864,0.963881,0.973158,0.980056,0.976595
0,HRPInception_thr3_v1_epoch309,0.960170,0.985847,0.963475,0.971934,0.980543,0.976219
0,HRPInception_thr3_v2_epoch428,0.961779,0.985811,0.963919,0.973582,0.979565,0.976564
0,HRPInception_thr3_v2_epoch342,0.959575,0.985735,0.963914,0.973247,0.979811,0.976518


In [14]:
label_acc = pickle.load(open("./PETA/HRPInception/thr3_v2_epoch462_valloss0.985864.hdf5_results.file", "rb"))['label_acc']
i = 0
with open("./PETA/HRPInception/thr3_attributes_list.txt", "r") as f:
    while True:
        lines = f.readline()
        if not lines:
            break
        print(lines, label_acc[i])
        i += 1

upperBodyLogo
 0.9138023630504833
lowerBodyThinStripes
 0.4999342105263158
upperBodyThinStripes
 0.9444499734607219
upperBodyThickStripes
 0.9997357643017571
accessoryHeadphone
 0.929205298013245
hairBald
 0.982886258144126
accessoryHairBand
 0.9561794769680074
accessoryHat
 0.9865646122646758
accessoryKerchief
 0.9633481270965942
hairLong
 0.9839816477863487
accessoryMuffler
 0.985404916738227
accessoryNothing
 0.9749626653321697
hairShort
 0.9793546840459431
accessorySunglasses
 0.9827457164033775
upperBodyJacket
 0.9608979348973712
upperBodyLongSleeve
 0.9735077775098546
upperBodyNoSleeve
 0.9563106796116505
upperBodyShortSleeve
 0.9761430314156155
upperBodySuit
 0.989142034463468
upperBodySweater
 0.9483399576832572
upperBodyTshirt
 0.981085171940528
upperBodyOther
 0.9758408786229025
upperBodyVNeck
 0.9410443003529723
lowerBodyHotPants
 0.9918686031861095
lowerBodyJeans
 0.9759291068312848
lowerBodyLongSkirt
 0.975986920755546
lowerBodyShorts
 0.9661483268797602
lowerBodyShortSkir